# Doctor Joins the Scottish Deanery

## Before they can become members of staff they must complete some pre employment checks

These include:

* Identity Verification (We think the GMC Credential + 1 Physical ID Document provides a strong Level of Assurance)
* Right to Work Check
* DBS Check
* GMC Licence

While at the deanery they can complete there compulsory basic training and receive the relevant immunity certificates

### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://healthcare-professional-agent:3021 and an api key of MyMedicalAPIKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)


print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [4]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))



connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)


def prover_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    
    
    if state == "request_received":
        presentation_request = payload["presentation_request"]
        print("Recieved Presentation Request\n")
        print("\nRequested Attributes - Note the restrictions. These limit the credentials we could respond with\n")
        print(presentation_request["requested_attributes"])
    elif state == "presentation_sent":
        print("The Presentation object is a bit overwhelming. Let's look at it in detail\n")
        
    elif state == "presentation_acked":
        print("Presentation has been acknowledged by the Issuer")
        
prover_listener = {
    "topic": "present_proof",
    "handler": prover_proof_handler
}

listeners.append(prover_listener)


agent_controller.register_listeners(listeners)

## Accept Invitation

Copy an invitation object from another agent playing the role inviter (see the inviter_template recipe)

In [5]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '5c322c3e-dddd-4711-82d4-d51b8adf7732', 'label': 'Scottish Deanery', 'recipientKeys': ['BnPZDJQPvd1ngtBfZ3HDsPmQ24sdeZ7ZemyDu73hTavf'], 'serviceEndpoint': 'https://136f0239772c.ngrok.io'}



In [6]:
auto_accept="false"
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  1f9c9783-ca0d-41be-9672-2845b2c8f0e6
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
invitation


In [7]:
# Label for the connection
my_label = None
# Endpoint you expect to recieve messages at
my_endpoint = None

accept_response = await agent_controller.connections.accept_invitation(connection_id, my_label, my_endpoint)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  1f9c9783-ca0d-41be-9672-2845b2c8f0e6
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  1f9c9783-ca0d-41be-9672-2845b2c8f0e6
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  1f9c9783-ca0d-41be-9672-2845b2c8f0e6
State :  active
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
Connection ID: 1f9c9783-ca0d-41be-9672-2845b2c8f0e6 is now active.

---------------------------------------------------------------------

Handle present-proof
Connection ID :  1f9c9

## Fetch Presentation Records

Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. This could also be done through the present_proof listeners which have access to a presentation record in the payload.

In [8]:
# Optional Query parameters
verifier_connection_id = connection_id
thread_id=None
state = "request_received"
role = "prover"

proof_records_response = await agent_controller.proofs.get_records(verifier_connection_id, thread_id, state, role)

# We fetch the first record from the response. You may want to customise this further
presentation_record = proof_records_response["results"][0]
presentation_exchange_id = presentation_record["presentation_exchange_id"]

## Search For Available Credentials to Construct Presentation From

The presentation record can be used to query your agents wallet and return all credentials that could be used to construct valid presentation

In [9]:
# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print("Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n")
# print(credentials)

credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}


# Note we are working on a friendlier api to abstract this away

if credentials:
    for credential in credentials:

        for attribute_name in credential["presentation_referents"]:
            if attribute_name not in credentials_by_reft:
                credentials_by_reft[attribute_name] = credential

for (key, value) in credentials_by_reft.items():
    print(f"Attribute {key} can be satisfied by Credential with Referent {value['cred_info']['referent']}")
                
for attribute_name in presentation_record["presentation_request"]["requested_attributes"]:
    if attribute_name in credentials_by_reft:
        revealed[attribute_name] = {
            "cred_id": credentials_by_reft[attribute_name]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }


print("\nGenerate the proof")
presentation = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}

Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal

Attribute GMC Number can be satisfied by Credential with Referent GMC
Attribute Name can be satisfied by Credential with Referent GMC
Attribute Headshot can be satisfied by Credential with Referent GMC

Generate the proof


## Send Presentation of GMC Licence Attributes

A presentation is sent in represent to a presentation record that has previously been created.

In [10]:
presentation_response = await agent_controller.proofs.send_presentation(presentation_exchange_id, presentation)


---------------------------------------------------------------------

Handle present-proof
Connection ID :  1f9c9783-ca0d-41be-9672-2845b2c8f0e6
Presentation Exchange ID :  4583c932-2f5f-4cf0-9b98-5f92dd44577a
Protocol State :  presentation_sent
Agent Role :  prover
Initiator :  external

---------------------------------------------------------------------

The Presentation object is a bit overwhelming. Let's look at it in detail



## Your Own Business Logic

Now you should have an established, active connection you can write any custom logic you want to engage with protocols with the connection

In [ ]:
## Custom Logic

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()